##### Ноутбук создан для попытки отказа от кластеризации в целом. 
Используется механизм строгих пороговых значений кластеров. 
Также, оценивается не волатильность, а реальные изменения цен между точками данных

In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
import pandas_profiling
from tqdm import tqdm
from copy import deepcopy

import cufflinks as cf
import plotly.offline
import plotly.express as px
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
path = '/home/vlad/HDD/GIT/CryptoLp DataScience/DataSets/Разные бары/'
file = '2019-2020_Bars_700T'
df_data = pd.read_csv(path+file+'.csv')
df_data = df_data.dropna()
df_data.index = range(len(df_data))

In [3]:
df_data.tail(3)

,datetime,open,low,high,close,volume
585697,2021-01-10 23:57:36.495287,38428.0,38300.0,38462.5,38300.0,4430463
585698,2021-01-10 23:59:30.593566,38300.0,38250.0,38357.5,38264.5,8412985
585699,2021-01-10 23:59:59.886955,38264.5,38225.0,38279.5,38225.0,5901463


In [4]:
# Вычисление стандартной волатильности
def calculation_volatility_STD(df_data_close_price:pd.Series,
                               win_size):
    volatility_STD = df_data_close_price.pct_change().rolling(win_size).std()*1000
    return volatility_STD

In [5]:
close = df_data['close']
volume = df_data['volume']

In [6]:
# %%time
# df_data['volatility 01'] = calculation_volatility_STD(df_data_close_price=df_data['close'], win_size=1)
# df_data['volatility 03'] = calculation_volatility_STD(df_data_close_price=df_data['close'], win_size=3)
# df_data['volatility 05'] = calculation_volatility_STD(df_data_close_price=df_data['close'], win_size=5)
# df_data['volatility 07'] = calculation_volatility_STD(df_data_close_price=df_data['close'], win_size=7)
# df_data['volatility 10'] = calculation_volatility_STD(df_data_close_price=df_data['close'], win_size=10)
# df_data['volatility 15'] = calculation_volatility_STD(df_data_close_price=df_data['close'], win_size=15)
# df_data['volatility 20'] = calculation_volatility_STD(df_data_close_price=df_data['close'], win_size=20)

In [7]:
df_data = df_data.dropna()
df_data.index = range(len(df_data))

In [8]:
df_data['mid price'] = (df_data['high'] + df_data['low'])/2
df_data['difference'] = df_data['high'] - df_data['low']

In [9]:
df_data.tail(2)

,datetime,open,low,high,close,volume,mid price,difference
3943351,2020-11-04 23:59:48.459946,14158.5,14158.5,14160.0,14158.5,397193,14159.25,1.5
3943352,2020-11-04 23:59:59.455549,14158.0,14151.5,14158.0,14151.5,291620,14154.75,6.5


In [10]:
thresholds = [0.0001, 0.00015, 0.0002, 0.00025, 0.0003]

In [11]:
for th in thresholds:
    df_data[f'th {th}'] = df_data['mid price']*th

In [12]:
for th in tqdm(thresholds):
    list_checks = list()
    for i in range(len(df_data)):
        if df_data[f'th {th}'].values[i] <= df_data['difference'].values[i]:
            list_checks.append(True)
        else:
            list_checks.append(False)
    df_data[f'mark_th {th}'] = list_checks

100%|██████████| 5/5 [01:38<00:00, 19.64s/it]


In [13]:
df_data.head(2)

,datetime,open,low,high,close,volume,mid price,difference,th 0.0001,th 0.00015,th 0.0002,th 0.00025,th 0.0003,mark_th 0.0001,mark_th 0.00015,mark_th 0.0002,mark_th 0.00025,mark_th 0.0003
0,2019-01-01 00:00:12.181624,3693.0,3693.0,3694.0,3694.0,220577,3693.50,1.0,0.369350,0.554025,0.73870,0.923375,1.108050,True,True,True,True,False
1,2019-01-01 00:00:19.741071,3694.0,3693.5,3694.0,3694.0,355202,3693.75,0.5,0.369375,0.554062,0.73875,0.923438,1.108125,True,False,False,False,False


In [14]:
# df_data['mark_th 0.0001'] = df_data['mark_th 0.0001'].map({True:'Extra low', False:'High',})
# df_data['mark_th 0.00015'] = df_data['mark_th 0.00015'].map({True:'Extra low', False:'High',})
# df_data['mark_th 0.0002'] = df_data['mark_th 0.0002'].map({True:'Extra low', False:'High',})
# df_data['mark_th 0.00025'] = df_data['mark_th 0.00025'].map({True:'Extra low', False:'High',})
# df_data['mark_th 0.0003'] = df_data['mark_th 0.0003'].map({True:'Extra low', False:'High',})

In [15]:
df_data = df_data[['datetime','open','low','high','close','volume','mid price','difference',
                   'mark_th 0.0001','mark_th 0.00015','mark_th 0.0002','mark_th 0.00025','mark_th 0.0003',
                  ]]

In [16]:
df_data.tail(2)

,datetime,open,low,high,close,volume,mid price,difference,mark_th 0.0001,mark_th 0.00015,mark_th 0.0002,mark_th 0.00025,mark_th 0.0003
3943351,2020-11-04 23:59:48.459946,14158.5,14158.5,14160.0,14158.5,397193,14159.25,1.5,True,False,False,False,False
3943352,2020-11-04 23:59:59.455549,14158.0,14151.5,14158.0,14151.5,291620,14154.75,6.5,True,True,True,True,True


In [17]:
df_data.to_csv('/home/brainiac/Vlad/Трэшхолды волатильности.csv',index=False)

## Окончание основнго блока

In [17]:
def cooman_superman(mark_th: pd.Series, window: int):
    return mark_th.rolling(window=window).sum().shift(1)

In [18]:
for column in ['mark_th 0.0001', 'mark_th 0.00015', 'mark_th 0.0002', 'mark_th 0.00025', 'mark_th 0.0003']:
    for window in [1, 2, 3, 4]:
        th = column.split('_')[1]
        series = cooman_superman(df_data[f'{column}'], window) 
        df_data[f'pred_{th}_{window}'] = series == series.max()

In [19]:
df_data.head(3)

,datetime,open,low,high,close,volume,mid price,difference,mark_th 0.0001,mark_th 0.00015,...,pred_th 0.0002_3,pred_th 0.0002_4,pred_th 0.00025_1,pred_th 0.00025_2,pred_th 0.00025_3,pred_th 0.00025_4,pred_th 0.0003_1,pred_th 0.0003_2,pred_th 0.0003_3,pred_th 0.0003_4
0,2019-01-01 00:00:12.181624,3693.0,3693.0,3694.0,3694.0,220577,3693.50,1.0,True,True,...,False,False,False,False,False,False,False,False,False,False
1,2019-01-01 00:00:19.741071,3694.0,3693.5,3694.0,3694.0,355202,3693.75,0.5,True,False,...,False,False,True,False,False,False,False,False,False,False
2,2019-01-01 00:00:19.741071,3694.0,3694.0,3694.5,3694.5,45796,3694.25,0.5,True,False,...,False,False,False,False,False,False,False,False,False,False


## Блок экспериментов с предсказанием режма

In [20]:
df_pred = deepcopy(df_data[['datetime','mid price',
                            'mark_th 0.0001','mark_th 0.00015','mark_th 0.0002','mark_th 0.00025','mark_th 0.0003',
                            'pred_th 0.0001_1','pred_th 0.0001_2','pred_th 0.0001_3','pred_th 0.0001_4',
                            'pred_th 0.00015_1','pred_th 0.00015_2','pred_th 0.00015_3','pred_th 0.00015_4',
                            'pred_th 0.0002_1', 'pred_th 0.0002_2','pred_th 0.0002_3','pred_th 0.0002_4',
                            'pred_th 0.00025_1','pred_th 0.00025_2','pred_th 0.00025_3','pred_th 0.00025_4',
                            'pred_th 0.0003_1','pred_th 0.0003_2','pred_th 0.0003_3','pred_th 0.0003_4',
                          ]])

In [21]:
df_pred['mark_th 0.0003'].value_counts(normalize=True)*100

False    68.283692
True     31.716308
Name: mark_th 0.0003, dtype: float64

In [22]:
th = 0.00025
df_regime = df_pred[['datetime','mid price','mark_th 0.00025',
                     f'pred_th {th}_1',f'pred_th {th}_2',f'pred_th {th}_3',f'pred_th {th}_4',
                    ]]

In [23]:
df_regime.head(2)

,datetime,mid price,mark_th 0.00025,pred_th 0.00025_1,pred_th 0.00025_2,pred_th 0.00025_3,pred_th 0.00025_4
0,2019-01-01 00:00:12.181624,3693.50,True,False,False,False,False
1,2019-01-01 00:00:19.741071,3693.75,False,True,False,False,False


In [24]:
df_regime[f'mark_th {th}'].value_counts(normalize=True)*100

False    59.861265
True     40.138735
Name: mark_th 0.00025, dtype: float64

In [25]:
from sklearn.metrics import accuracy_score

accuracy_score(df_regime[f'mark_th {th}'].values, df_regime[f'pred_th {th}_1'].values)
accuracy_score(df_regime[f'mark_th {th}'].values, df_regime[f'pred_th {th}_2'].values)
accuracy_score(df_regime[f'mark_th {th}'].values, df_regime[f'pred_th {th}_3'].values)
accuracy_score(df_regime[f'mark_th {th}'].values, df_regime[f'pred_th {th}_4'].values)

0.7080568237233643

0.7063980830526712

0.6963363411797017

0.6849328477567187

### Окнчание блока эксперимента

### Блок разметки по волатильности

In [26]:
df_volat = df_data[['datetime','mid price','difference',]]

In [27]:
def calculation_volatility_STD(df_data_close_price:pd.Series,
                               win_size):
    volatility_STD = df_data_close_price.pct_change().rolling(win_size).std()*1000
    return volatility_STD

In [28]:
df_volat.head(3)

,datetime,mid price,difference
0,2019-01-01 00:00:12.181624,3693.50,1.0
1,2019-01-01 00:00:19.741071,3693.75,0.5
2,2019-01-01 00:00:19.741071,3694.25,0.5


In [29]:
df_volat['volatility 2'] = calculation_volatility_STD(df_volat['mid price'], 2)
df_volat['volatility 3'] = calculation_volatility_STD(df_volat['mid price'], 3)
df_volat['volatility 5'] = calculation_volatility_STD(df_volat['mid price'], 5)
df_volat['volatility 7'] = calculation_volatility_STD(df_volat['mid price'], 7)
df_volat['volatility 10'] = calculation_volatility_STD(df_volat['mid price'], 10)

In [30]:
df_volat = df_volat.dropna()
df_volat.index = range(len(df_volat))
df_volat.head()

,datetime,mid price,difference,volatility 2,volatility 3,volatility 5,volatility 7,volatility 10
0,2019-01-01 00:06:47.504952,3688.50,1.0,0.095788,0.067732,0.113336,0.169047,0.199289
1,2019-01-01 00:06:58.148842,3687.00,2.0,0.191719,0.135566,0.151546,0.176670,0.202004
2,2019-01-01 00:07:16.755020,3685.75,1.5,0.047829,0.141124,0.107214,0.134755,0.172516
3,2019-01-01 00:09:06.675608,3684.75,0.5,0.047881,0.067677,0.100554,0.134776,0.152436
4,2019-01-01 00:09:59.772810,3685.00,1.0,0.239824,0.218008,0.188142,0.155171,0.181325


In [31]:
%%time
target = list()
for i in tqdm(range(len(df_volat))):
    if df_volat['difference'][i] <= 1.5:
        target.append('low')
    else:
        target.append('high')
df_volat['target'] = target

100%|██████████| 3943343/3943343 [01:12<00:00, 54281.98it/s]

CPU times: user 1min 12s, sys: 132 ms, total: 1min 12s
Wall time: 1min 12s


In [32]:
df_volat.head(3)

,datetime,mid price,difference,volatility 2,volatility 3,volatility 5,volatility 7,volatility 10,target
0,2019-01-01 00:06:47.504952,3688.50,1.0,0.095788,0.067732,0.113336,0.169047,0.199289,low
1,2019-01-01 00:06:58.148842,3687.00,2.0,0.191719,0.135566,0.151546,0.176670,0.202004,high
2,2019-01-01 00:07:16.755020,3685.75,1.5,0.047829,0.141124,0.107214,0.134755,0.172516,low


In [33]:
df_volat['volatility 3'].describe()

count    3.943343e+06
mean     1.658611e-01
std      3.040573e-01
min      0.000000e+00
25%      4.557686e-02
50%      9.349331e-02
75%      1.927651e-01
max      8.764917e+01
Name: volatility 3, dtype: float64

In [34]:
df_volat['volatility 3'].quantile(0.3)

0.05393389932573242

In [39]:
th = df_volat['volatility 10'].quantile(0.3)
df_volat[df_volat['volatility 10']<=th]['target'].value_counts(normalize=True)*100

low     83.754986
high    16.245014
Name: target, dtype: float64

In [13]:
%%time
windows = ['2','3','5','7','10',
          ]
for win in windows:
    anomalies = list()
    volatility = df_data[f'volatility {win}']
    lbord, rbord = df_data[f'volatility {win}'].quantile(0.10), df_data[f'volatility {win}'].quantile(0.90)
    for outlier in volatility:
        if outlier > rbord:
            anomalies.append('extreme high')
        elif outlier < lbord:
            anomalies.append('extreme low')
        else:
            anomalies.append('normal')
    df_data[f'outlier {win}'] = anomalies

CPU times: user 8.33 s, sys: 1.96 ms, total: 8.33 s
Wall time: 8.33 s


In [16]:
df_data.head(3)

,datetime,open,low,high,close,volume,volatility 03,volatility 05,volatility 07,volatility 10,volatility 15,volatility 20,outlier 03,outlier 05,outlier 07,outlier 10,outlier 15,outlier 20,difference
0,2019-01-01 00:19:44.218019,3689.0,3687.5,3689.0,3687.5,303729,0.234759,0.287578,0.310358,0.334924,0.284433,0.289274,normal,normal,normal,normal,normal,normal,1.5
1,2019-01-01 00:20:41.231962,3687.5,3686.5,3687.5,3686.5,224532,0.207050,0.191699,0.338697,0.296085,0.289320,0.291280,normal,normal,normal,normal,normal,normal,1.0
2,2019-01-01 00:23:06.890890,3686.5,3686.0,3686.5,3686.5,160235,0.207050,0.191699,0.256208,0.279017,0.289320,0.287027,normal,normal,normal,normal,normal,normal,0.5


In [47]:
len(df_data)
len(df_data[df_data['difference'] <= 1])

3888875

1713770

In [48]:
1713770/3888875

0.44068528816174346

In [18]:
len(df_data)
len(df_data[df_data['outlier 03']=='extreme low'])
len(df_data[df_data['target']=='low'])

3888875

388888

2143355

In [19]:
df_result = deepcopy(df_data[['datetime', 'close',
                              'outlier 03', 
                              'outlier 05', 'outlier 07','outlier 10', 'outlier 15', 'outlier 20',
                              'target'
                             ]]
                    )

In [38]:
df_month = df_result[(df_result['datetime']>="2020-06-01") & (df_result['datetime']<"2020-10-01")]

In [39]:
df_month['target'].value_counts(normalize=True)*100

high    55.073955
low     44.926045
Name: target, dtype: float64

In [43]:
df_month[df_month[f'outlier 05'] == 'extreme low']['target'].value_counts(normalize=True)*100

low     84.990361
high    15.009639
Name: target, dtype: float64

In [23]:
df_month.head(1)

,datetime,close,outlier 03,outlier 05,outlier 07,outlier 10,outlier 15,outlier 20,target
3654950,2020-08-01 00:00:04.926663,11362.0,normal,normal,normal,normal,normal,normal,high


In [33]:
len(df_month)
len(df_month[df_month[f'outlier 05'] == 'extreme low'])

df_month[df_month[f'outlier 10'] == 'extreme low']['target'].value_counts(normalize=True)*100

108523

5034

low     70.502336
high    29.497664
Name: target, dtype: float64

In [17]:
df_data[3655100-3:3655100+2]

,datetime,open,low,high,close,volume,volatility 03,volatility 05,volatility 07,volatility 10,volatility 15,volatility 20,outlier 03,outlier 05,outlier 07,outlier 10,outlier 15,outlier 20
3655097,2020-08-01 00:26:58.021986,11291.5,11291.5,11295.0,11295.0,477819,0.044262,0.200560,0.200489,0.360328,0.448362,0.453464,normal,normal,normal,normal,normal,normal
3655098,2020-08-01 00:27:47.451965,11295.0,11294.5,11295.0,11295.0,293199,0.193060,0.152147,0.200489,0.312971,0.415780,0.440180,normal,normal,normal,normal,normal,normal
3655099,2020-08-01 00:28:26.680962,11295.0,11294.5,11295.0,11295.0,342774,0.178960,0.172689,0.185177,0.319474,0.398057,0.430567,normal,normal,normal,normal,normal,normal
3655100,2020-08-01 00:28:56.482583,11294.5,11294.5,11295.0,11295.0,413645,0.000002,0.182614,0.159116,0.186166,0.386760,0.417573,extreme low,normal,normal,normal,normal,normal
3655101,2020-08-01 00:29:55.909229,11295.0,11294.5,11295.0,11295.0,240320,0.000002,0.138622,0.167695,0.186166,0.355567,0.410724,extreme low,normal,normal,normal,normal,normal


### Построение графика

In [19]:
# Построение графиков
def show_plot_plotly(df_data:pd.DataFrame,
                     left_border,
                     right_border,
                     x_,
                     y_,
                     color_:str,
                     title_:str,             
             ):  
    fig = px.scatter(df_data[left_border:right_border], 
                 x=x_, 
                 y=y_, 
                 color=color_, 
                 color_discrete_sequence=['dodgerblue','limegreen','tomato'],
                 title=title_
                )    
    return fig

In [20]:
df_result.head(2)

,datetime,close,outlier 03,outlier 05,outlier 07,outlier 10,outlier 15,outlier 20
0,2019-01-01 00:19:44.218019,3687.5,normal,normal,normal,normal,normal,normal
1,2019-01-01 00:20:41.231962,3686.5,normal,normal,normal,normal,normal,normal


In [21]:
win = '03'
df_month = df_result[(df_result['datetime']>="2020-08-01") & (df_result['datetime']<"2020-09-01")]

fig = show_plot_plotly(df_data=df_month,
                       left_border = 0,
                       right_border=len(df_month),
                       x_='datetime',
                       y_='close',
                       color_=f'outlier {win}',
                       title_=f'outlier {win}',
                      )
plotly.offline.plot(fig,filename=f'/home/brainiac/Vlad/Fixed Volat {win}.html')

'/home/brainiac/Vlad/Fixed Volat 03.html'

In [41]:
# df_result[(df_result['datetime']>="2020-05-01") & 
#           (df_result['datetime']<"2020-06-01")][['datetime','regimes']].to_csv('/home/brainiac/Vlad/2020_05.csv',index=False)

In [24]:
df_month = df_result[(df_result['datetime']>="2020-08-01") & (df_result['datetime']<"2020-09-01")]
df_month.head(1)

,datetime,close,outlier 03,outlier 05,outlier 07,outlier 10,outlier 15,outlier 20
522119,2020-08-01 00:00:36.205894,11372.5,normal,normal,normal,normal,normal,normal


In [22]:
len(df_month[df_month['outlier 03']=='normal'])
len(df_month[df_month['outlier 03']=='extreme low'])

106448

857

In [24]:
125/15083*100

0.8287475966319697

In [25]:
857/106448*100

0.805087930257027

In [26]:
0.8287475966319697 - 0.805087930257027

0.023659666374942767